<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;">
              <h1>
               <left>Determing Growth Rate</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: ASU CBP Summer School 2025
Instructor: Dr. Douglas Shepherd
Contact Info: douglas.shepherd@asu.edu
Authors: Dr. Douglas Shepherd
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2018 Griffin Chure

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/M4B_Determining_Growth_Rate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/M4B_Determining_Growth_Rate.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Learning Objectives
In this lesson, we are going to analyze and fit experimental data of bacteria dividing to determine their growth rate. After completing this lesson, you should be able to:
* Determine the distribution of cell division times from experimental data
* Fit the population growth to the exponential growth equation

# Using microscopy to measure bacterial growth

While we have just scraped the surface of growth in class, there is a seemingly endless reservoir of literature that explores the fascinating physiology of cellular growth. For a detailed summary, see [this review](https://jun.ucsd.edu/files/publications/Jun_ROPP_2018.pdf) by Suckjoon Jun and colleagues which gives both a historical accounting of bacterial growth physiology and a view of the future of the field.

As discussed in class and examined numerically in [Module 4A](M4A_ExponentialGrowth.ipynb), the continuous growth phase of a bacterial culture can be stated mathematically using the Exponetial Growth equation,

$$
N(t) = N_0 e^{rt}, \tag{1}
$$

where $N(t)$ is the number of cells at time $t$, $N_0$ is the initial number of cells and $r$ is the growth rate in dimensions of time.   Experimentally, it's difficult to count the number of cells in a large culture, but easy to measure for smaller colonies and cultures.

In this tutorial, we will use a movie of *E. coli* cells growing on a hard agar subtstrate at 100x magnification. Below, we have raw phase microscopy data (left), segmentation of individual cells (center), and the result of tracking individual cells and daughter cells (right).

<p style="text-align:center;">
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/Figures/data_gif.gif?raw=1" width="200" alt="Raw E. coli phase data" /><br>
    <strong>Phase data</strong>
  </span>
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/Figures/mask_gif.gif?raw=1" width="200" alt="Segemented E. coli phase data" /><br>
    <strong>Segmented</strong>
  </span>
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/Figures/tracked_gif.gif?raw=1" width="200" alt="Tracked E. Coli phase data" /><br>
    <strong>Tracked</strong>
  </span>
</p>

# More about microscopy data

While we often think of images as a colorful photograph, it is important to remember that at its core, a digital image is nothing but data. An image is a matrix of numbers upon which we can deploy mathematical operations. We'll download the data underlying the above movies to your Colab instance in the next code block.

In [ ]:
# Download data
!pip install -q gdown

url = "https://drive.google.com/uc?id=1nxY9-jEGmrH8elRdiMENFhBb64mMYl8q"

# download the .zip
!gdown "{url}" -O /content/cbpSS2025.zip

# unzip as before
!unzip -q /content/cbpSS2025.zip -d /content/cbpSS2025/

Downloading...
From (original): https://drive.google.com/uc?id=1nxY9-jEGmrH8elRdiMENFhBb64mMYl8q
From (redirected): https://drive.google.com/uc?id=1nxY9-jEGmrH8elRdiMENFhBb64mMYl8q&confirm=t&uuid=03183d3d-d82a-48b8-b930-352ee066064d
To: /content/cbpSS2025.zip
100% 32.0M/32.0M [00:00<00:00, 50.9MB/s]
replace /content/cbpSS2025/masks/Composite_00_cp_masks.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Viewing "n" dimensional data in Python

We can view these timelapse data using an interactive viewer, such as [stackview](https://pypi.org/project/stackview/). To do this, we need to know a little more about the experiment itself. We often call this "metadata" and the information is critical to correctly analyzing experiments.

The camera used in these experiments has a pixel size of $6.5$ $\mu m$.
The objective used in the experiments has a magnficiation of $100$ $\times$.
The interval between pictures is $5$ minutes.

To know how big each pixel in our digital images is, we divide the camera pixel size by the microscope magnfication, $6.5/100$ $=$ $.065$ $\mu m$.

In [ ]:
# install pacakges for loading and viewing image data in colab
!pip install stackview==0.16.0 ipycanvas==0.11 numcodecs imagecodecs tifffile

In [ ]:
# import packages for loading and viewing image data in colab
from google.colab import output
output.enable_custom_widget_manager()

from pathlib import Path
import tifffile
import numpy as np
import stackview

## Raw data

We will first look at the raw data, which is acquired with [phase-contrast microscopy](https://www.ibiology.org/talks/phase-contrast-microscopy/) and [fluorescence microscopy](https://www.ibiology.org/talks/fluorescence-microscopy/). Phase-contrast microscopy is method that enchances the visibility of biological structures by manipulating the "phase" of the light passing through the sample. In this method, the bacteria cell bodies are darker than the surrounding environment. Fluorescence microscopy is a method that excites and collects light from fluorophores that are present in the bacteria. In this method, the bacteria cell bodies are brighter than the surrounding environment.

In [ ]:
# load and view raw phase and fluorescence image data

raw_files = sorted(Path("/content/cbpSS2025/data/").glob("*.tif*"))
raw_data = []
for raw_file in raw_files:
  raw_data.append(tifffile.imread(raw_file))
raw_data = np.asarray(raw_data,dtype=np.uint16)

stackview.slice(raw_data)

## Segmented phase-contrast data

The phase-contrast data has been passed through [cellpose](https://github.com/MouseLand/cellpose), a deep-learning Python package that excels at automatically finding cells. You can try cellpose out online for free [here](https://huggingface.co/spaces/mouseland/cellpose).

Starting from raw data, cellpose automatically finds unique cells and assigns all pixels within each cell a unique integer indentifier. This is called a mask image because it creates a unique mask for each pixel. A **critical** note here is that each image is segmented independently, so the mask identifier resets in each image. That means we can use this data to count the number of cells present at each time point.

In [ ]:
# load and view segmented phase-contrast image data

mask_files = sorted(Path("/content/cbpSS2025/masks/").glob("*.tif*"))
mask_data = []
for mask_file in mask_files:
  mask_data.append(tifffile.imread(mask_file))
mask_data = np.asarray(mask_data,dtype=np.uint16)

stackview.slice(mask_data)

## Tracked phase-contrast data

While the segmented data is helpful to easily get the number of cells, we could extract more information from this simple experiment by quantifying the timing for cell divisions and cell lineages. To do this, we need the identifiers to stay the same for each cell and then mark when a cell division occurs. To do this, we can track each cell and it's daughter cells using [trackastra](https://github.com/weigertlab/trackastra), a deep-learning Python package that excels at tracking cells.

Starting from raw data and the segmentation masks generated by cellpose, trackastra automatically tracks cells across time and tracks cell division. This small changes in how the data is processed will allow us to later extract the time between cell division for our bacteria colony!

In [ ]:
# load and view segmented and tracked phase-contrast image data

tracked_files = sorted(Path("/content/cbpSS2025/result/").glob("*.tif*"))
tracked_data = []
for tracked_file in tracked_files:
  tracked_data.append(tifffile.imread(tracked_file))
tracked_data = np.asarray(tracked_data,dtype=np.uint16)

stackview.slice(tracked_data)

# Multiple ways to analyze this experiment

Using the above experimental and processed data, here are four ways you could analyze the dynamics of bacteria growth.

1. Quantify the total area of bacteria per time and see if it follows the Exponential Growth or Logistic Growth equation.
2. Quanify the number of bacteria per time and see if it follows the Exponential Growth or Logistic Growth equation.
3. Quantify the mean time between cell divisions and compare it to the rate obtained using 1 and 2 above.
4. Quantify the full distribution of time between cell division across the entire dataset. Does it match your expectations?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

In [ ]:
first_frame = mask_data[0]

unique_ids = np.unique(first_frame)
initial_cells = len(unique_ids) - 1

print(f"The number of starting cells from the image data is: {initial_cells}")

In [ ]:
N_0 = 8
r = 0.03
total_time = 150
delta_t = 0.1
n_time_steps = int(total_time / delta_t)

N_t = np.zeros(n_time_steps)
N_t[0] = N_0

for t in range(1, n_time_steps):

    dN = N_t[t - 1] * r * delta_t
    N_t[t] = N_t[t - 1] + dN

time_range = np.arange(0, total_time, delta_t)
plt.plot(time_range, N_t, 'b-', label='numerical integration')
plt.xlabel('time [min]')
plt.ylabel('number of cells')
plt.legend()

In [ ]:
# Initialize an empty list to store the calculated areas
bacterial_area = []

# Iterate through each frame in the mask_data array
for frame in mask_data:
    # Calculate the number of pixels that are not equal to the background value (assuming background is 0)
    area = np.sum(frame > 0)
    # Append the calculated area for the current frame to the bacterial_area list
    bacterial_area.append(area)

# Optionally print the first few elements of the bacterial_area list
print(f"Calculated bacterial area per frame (first 10): {bacterial_area[:10]}...")

In [ ]:
# Determine the total number of frames
num_frames = mask_data.shape[0]

# Define the time interval between frames (in minutes)
time_interval = 5

# Create a time array
time_array = np.arange(num_frames) * time_interval

# Print the number of frames, the time interval, and the first few elements of the generated time_array
print(f"Number of frames: {num_frames}")
print(f"Time interval: {time_interval} minutes")
print(f"First 10 time points: {time_array[:10]} minutes")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(time_array, bacterial_area)
plt.xlabel('Time [min]')
plt.ylabel('Bacterial Area [pixels]')
plt.title('Bacterial Area Over Time')
plt.show()

In [ ]:
def exponential_growth(t, N0, r):
  return N0 * np.exp(r * t)

def logistic_growth(t, N0, r, K):
  return K / (1 + (K / N0 - 1) * np.exp(-r * t))

In [ ]:
from scipy.optimize import curve_fit

# Use curve_fit to find the optimal parameters (N0 and r) for exponential growth
params_exp, covariance_exp = curve_fit(exponential_growth, time_array, bacterial_area)

# Extract the fitted parameters
N0_fit_exp, r_fit_exp = params_exp

# Print the fitted values
print(f"Fitted initial bacterial area (N0) for exponential growth: {N0_fit_exp:.2f}")
print(f"Fitted growth rate (r) for exponential growth: {r_fit_exp:.4f}")

In [ ]:
# Use curve_fit to find the optimal parameters (N0 and r) with an initial guess for N0
initial_area = bacterial_area[0]
initial_guess_exp = [initial_area, 0.03] # Use initial_area from the first frame and a reasonable guess for r
params_exp, covariance_exp = curve_fit(exponential_growth, time_array, bacterial_area, p0=initial_guess_exp)

# Extract the fitted parameters
N0_fit_exp, r_fit_exp = params_exp

# Print the fitted values
print(f"Fitted initial bacterial area (N0) for exponential growth: {N0_fit_exp:.2f}")
print(f"Fitted growth rate (r) for exponential growth: {r_fit_exp:.4f}")

In [ ]:
# Generate time values for the fitted curve
time_fit = np.linspace(0, time_array.max(), 100)

# Calculate the predicted bacterial area using the fitted parameters
bacterial_area_fit = exponential_growth(time_fit, N0_fit_exp, r_fit_exp)

# Create the plot
plt.figure(figsize=(8, 6))

# Plot experimental data
plt.scatter(time_array, bacterial_area, label='Experimental Data')

# Plot the fitted curve
plt.plot(time_fit, bacterial_area_fit, color='red', label=f'Exponential Fit ($N_0$={N0_fit_exp:.2f}, r={r_fit_exp:.4f})')

# Add labels and title
plt.xlabel('Time [min]')
plt.ylabel('Bacterial Area [pixels]')
plt.title('Bacterial Area with Exponential Fit')

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
from scipy.optimize import curve_fit

# Use curve_fit to find the optimal parameters (N0, r, and K) for logistic growth
initial_area = bacterial_area[0]
max_area = max(bacterial_area)
initial_guess_logistic = [initial_area, 0.03, max_area * 2] # Use initial_area, a reasonable guess for r, and a guess for K
params_logistic, covariance_logistic = curve_fit(logistic_growth, time_array, bacterial_area, p0=initial_guess_logistic)

# Extract the fitted parameters
N0_fit_logistic, r_fit_logistic, K_fit_logistic = params_logistic

# Print the fitted values
print(f"Fitted initial bacterial area (N0) for logistic growth: {N0_fit_logistic:.2f}")
print(f"Fitted growth rate (r) for logistic growth: {r_fit_logistic:.4f}")
print(f"Fitted carrying capacity (K) for logistic growth: {K_fit_logistic:.2f}")

In [ ]:
# Generate time values for the fitted logistic curve
time_fit = np.linspace(0, time_array.max(), 100)

# Calculate the predicted bacterial area using the fitted logistic parameters
bacterial_area_fit_logistic = logistic_growth(time_fit, N0_fit_logistic, r_fit_logistic, K_fit_logistic)

# Create the plot
plt.figure(figsize=(8, 6))

# Plot experimental data
plt.scatter(time_array, bacterial_area, label='Experimental Data')

# Plot the fitted logistic curve
plt.plot(time_fit, bacterial_area_fit_logistic, color='green', label=f'Logistic Fit ($N_0$={N0_fit_logistic:.2f}, r={r_fit_logistic:.4f}, K={K_fit_logistic:.2e})')

# Add labels and title
plt.xlabel('Time [min]')
plt.ylabel('Bacterial Area [pixels]')
plt.title('Bacterial Area with Logistic Fit')

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
from sklearn.metrics import r2_score

# Generate time values for the fitted curves (using the same range for both)
time_fit = np.linspace(0, time_array.max(), 100)

# Calculate the predicted bacterial area using the fitted exponential parameters
bacterial_area_fit_exp_plot = exponential_growth(time_fit, N0_fit_exp, r_fit_exp)

# Calculate the predicted bacterial area using the fitted logistic parameters
bacterial_area_fit_logistic_plot = logistic_growth(time_fit, N0_fit_logistic, r_fit_logistic, K_fit_logistic)

# Create the plot
plt.figure(figsize=(10, 7))

# Plot experimental data
plt.scatter(time_array, bacterial_area, label='Experimental Data')

# Plot the fitted exponential curve
plt.plot(time_fit, bacterial_area_fit_exp_plot, color='red', label=f'Exponential Fit ($r$={r_fit_exp:.4f})')

# Plot the fitted logistic curve
plt.plot(time_fit, bacterial_area_fit_logistic_plot, color='green', label=f'Logistic Fit ($r$={r_fit_logistic:.4f}, K={K_fit_logistic:.2e})')

# Add labels and title
plt.xlabel('Time [min]')
plt.ylabel('Bacterial Area [pixels]')
plt.title('Bacterial Area Growth: Exponential vs. Logistic Fit')

# Add legend
plt.legend()

# Display the plot
plt.show()

# Calculate R-squared for exponential fit
# Need to predict values at the actual time points for R-squared calculation
bacterial_area_pred_exp = exponential_growth(time_array, N0_fit_exp, r_fit_exp)
r2_exp = r2_score(bacterial_area, bacterial_area_pred_exp)

# Calculate R-squared for logistic fit
bacterial_area_pred_logistic = logistic_growth(time_array, N0_fit_logistic, r_fit_logistic, K_fit_logistic)
r2_logistic = r2_score(bacterial_area, bacterial_area_pred_logistic)

# Print R-squared values
print(f"R-squared for Exponential Fit: {r2_exp:.4f}")
print(f"R-squared for Logistic Fit: {r2_logistic:.4f}")